---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [3]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);
        

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [30]:
def answer_one():
        
    # Your Code Here
    Data1 = pd.read_csv('Employee_Movie_Choices.txt', sep = '\t')
    Data1.columns = ['Employee','Movie']
    
    L1 = nx.from_pandas_dataframe(Data1, 'Employee', 'Movie')
    
    return L1

answer_one()
#plot_graph(answer_one(), weight_name = None)


### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [26]:
def answer_two():
    
    # Your Code Here
    L1 = answer_one()
    Nodes_ = L1.nodes()
    
    for nodes in Nodes_:
        if nodes in employees:
            L1.add_node(nodes, type = 'employee')
        else:
            L1.add_node(nodes, type = 'movie')
    
    return L1

answer_two()
#plot_graph(answer_two(), weight_name = None)


### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [29]:
def answer_three():
        
    # Your Code Here
    L2 = answer_two()    
    G2 = bipartite.weighted_projected_graph(L2, employees)
    
    return G2

answer_three()
#plot_graph(answer_three(), weight_name = None)


### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [92]:
def answer_four():
        
    # Your Code Here
    Data1 = pd.read_csv('Employee_Movie_Choices.txt', sep = '\t')
    Data1.columns = ['Employee','Movie']

    Df1 = Data1.groupby(by = 'Employee').agg({'Movie': lambda x: ','.join(x)})
    Df1['Movie'] = [Mov_list.split(',') for Mov_list in Df1['Movie']]
    
    Data2 = pd.read_csv('Employee_Relationships.txt', sep = '\t', header = None)
    Data2.columns = ['Employee1', 'Employee2', 'Score']

    Df2 = Data2
    Df2['List1'] = [Df1.loc[name][0] for name in Df2['Employee1']]
    Df2['List2'] = [Df1.loc[name][0] for name in Df2['Employee2']]

    Df3 = Df2
    Df3['Mutual_movies'] = [len(list(set(A) & set(B))) for (A,B) in zip(Df3['List1'], Df3['List2'])]

    Df_final = Df3[['Employee1','Employee2','Score','Mutual_movies']]
    sol = Df_final['Score'].corr(Df_final['Mutual_movies'])
    
    return sol

answer_four()


0.78839622217334748